# using the ENWL representative feeders in PowerModelsDistribution.jl 
This is a notebook that demonstrates how to load the representative feeders of the ENWL data set using PowerModelsDistribution.jl and perform basic operations.

In [15]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\mnumair\OneDrive - KU Leuven\PhD Agenda\2024\24-10 October\TSK-467 RepoRepFed\Representative-distribution-networks\RF_ENWL_representative_feeders\PMD`


In [16]:
using PowerModelsDistribution
using Ipopt
using FileIO

Here you can choose any of the 4 representative feeders of and load it as an ENGINEERING model of PowerModelsDistribution.jl using the `load()` with the file name as argument. below an example for loading the first representative feeder.

In [17]:
RF1 = load("ENWL_RF1.jld2")

Dict{String, Any} with 10 entries:
  "voltage_source" => Dict{String, Any}("source"=>Dict{String, Any}("source_id"…
  "name"           => "ntw_2_fdr_5"
  "line"           => Dict{String, Any}("line96"=>Dict{String, Any}("length"=>0…
  "conductor_ids"  => [1, 2, 3, 4]
  "settings"       => Dict{String, Any}("sbase_default"=>100000.0, "vbases_defa…
  "files"          => ["C:/Users/mnumair/OneDrive - KU Leuven/PhD Files/2- Repo…
  "load"           => Dict{String, Any}("load2"=>Dict{String, Any}("source_id"=…
  "bus"            => Dict{String, Any}("109"=>Dict{String, Any}("lat"=>273.787…
  "linecode"       => Dict{String, Any}("3c_300_sac_xc"=>Dict{String, Any}("b_f…
  "data_model"     => ENGINEERING

from the ENGINEERING model `RF1` it is possible to explore the network:
For example, lookup the linecode for the first line in the network:

In [18]:
line1_lc = RF1["line"]["line1"]["linecode"]

"3c_300_sac_xc"

Then we can find the details for that linecode from the `linecode` dictionary:

In [22]:
RF1["linecode"][line1_lc] 

Dict{String, Any} with 7 entries:
  "b_fr"  => [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
  "rs"    => [0.000266 0.000164 0.000164; 0.000164 0.000266 0.000164; 0.000164 …
  "cm_ub" => [600.0, 600.0, 600.0]
  "xs"    => [8.36667e-5 1.06667e-5 1.06667e-5; 1.06667e-5 8.36667e-5 1.06667e-…
  "b_to"  => [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
  "g_to"  => [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
  "g_fr"  => [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]

and can see the formatted series resistance matrix for that linecode:

In [23]:
RF1["linecode"][line1_lc]["rs"]

3×3 Matrix{Float64}:
 0.000266  0.000164  0.000164
 0.000164  0.000266  0.000164
 0.000164  0.000164  0.000266

However most of the interesting analysis happens using the MATHEMATICAL model which is used for optimization problems. So we have to transform `RF1` from the ENGINEERING data model to the MATHEMATICAL model `RF1_math` 

In [20]:
RF1_math = transform_data_model(RF1)

Dict{String, Any} with 18 entries:
  "is_kron_reduced" => true
  "conductor_ids"   => [1, 2, 3]
  "time_elapsed"    => 1.0
  "bus"             => Dict{String, Any}("306"=>Dict{String, Any}("vm_pair_lb"=…
  "name"            => "ntw_2_fdr_5"
  "map"             => Dict{String, Any}[Dict("unmap_function"=>"_map_math2eng_…
  "settings"        => Dict{String, Any}("sbase_default"=>100000.0, "vbases_def…
  "gen"             => Dict{String, Any}("1"=>Dict{String, Any}("pg"=>[0.0, 0.0…
  "branch"          => Dict{String, Any}("306"=>Dict{String, Any}("br_r"=>[3.76…
  "storage"         => Dict{String, Any}()
  "switch"          => Dict{String, Any}()
  "is_projected"    => true
  "per_unit"        => true
  "data_model"      => MATHEMATICAL
  "shunt"           => Dict{String, Any}()
  "transformer"     => Dict{String, Any}()
  "bus_lookup"      => Dict{Any, Int64}("153"=>308, "54"=>2, "101"=>3, "41"=>4,…
  "load"            => Dict{String, Any}("4"=>Dict{String, Any}("model"=>POWER,…

Then it is possible to perform an optimal power flow (OPF) using `PowerModelsDistribution.jl` functions

In [32]:
RF1_opf = solve_mc_opf(RF1_math, ACPUPowerModel, Ipopt.Optimizer)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:    44853
Number of nonzeros in inequality constraint Jacobian.:     9222
Number of nonzeros in Lagrangian Hessian.............:   111474

Total number of variables............................:     5553
                     variables with only lower bounds:      924
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     5553
Total number of inequality c

Dict{String, Any} with 8 entries:
  "solve_time"         => 3.003
  "optimizer"          => "Ipopt"
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 0.0232051
  "solution"           => Dict{String, Any}("branch"=>Dict{String, Any}("306"=>…
  "objective_lb"       => -Inf

Then we can explore the results from the `"solution"` dictionary. FOr example we can find the voltage magnitude `vm` and angle `va` for the three phases of the first bus as follows

In [33]:
RF1_opf["solution"]["bus"]["1"]

Dict{String, Any} with 2 entries:
  "va" => [-0.0103246, -2.10709, 2.08696]
  "vm" => [0.98402, 0.986005, 0.992666]

### Notes

NOTE 1: All of the above can be done for any of the 4 representative feeders of the ENWL data set, bus simply swapping the file path in the `load()` function.

NOTE 2: This is just an example of how to load the ENWL representative feeders in PowerModelsDistribution.jl. For more information on how to use PowerModelsDistribution.jl package, please refer to the [documentation](https://lanl-ansi.github.io/PowerModelsDistribution.jl/stable/).
